In [1]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.morphology import skeletonize, remove_small_objects
from skimage.measure import label, regionprops


In [ ]:

# 1. Load and binarize
img = cv2.imread('/home/aaron/f110_gymnasium_ros2_jazzy/rl_training/maps/Shanghai_map.png', cv2.IMREAD_GRAYSCALE)
_, binary = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
if np.sum(binary == 255) < np.sum(binary == 0):
    binary = cv2.bitwise_not(binary)
mask = (binary // 255).astype(bool)


In [ ]:

# 2. Skeletonize
skeleton = skeletonize(mask)
skeleton = remove_small_objects(skeleton, min_size=100)


In [ ]:

# 3. Get all skeleton points
ys, xs = np.where(skeleton)
points = np.stack([xs, ys], axis=1)


In [ ]:

# 4. Find the largest connected component (main loop)
labeled = label(skeleton)
regions = sorted(regionprops(labeled), key=lambda r: r.area, reverse=True)
main_label = regions[0].label
main_skeleton = (labeled == main_label)
ys, xs = np.where(main_skeleton)
centerline_points = np.stack([xs, ys], axis=1)


In [ ]:

# 5. (Optional) Order the points for a continuous path (simple nearest-neighbor chaining)
from scipy.spatial import cKDTree

def order_skeleton_points(points):
    ordered = [points[0]]
    used = set([0])
    kdtree = cKDTree(points)
    for _ in range(1, len(points)):
        dists, idxs = kdtree.query(ordered[-1], k=min(10, len(points)))
        if np.isscalar(idxs): idxs = [idxs]
        found = False
        for idx in idxs:
            if idx not in used:
                ordered.append(points[idx])
                used.add(idx)
                found = True
                break
        if not found:
            break
    return np.array(ordered)

ordered_centerline = order_skeleton_points(centerline_points)


In [ ]:

# 6. Save as CSV
df = pd.DataFrame(ordered_centerline, columns=['x', 'y'])
df.to_csv('ordered_centerline.csv', index=False)
print(f"Ordered centerline points saved to ordered_centerline.csv (count: {len(ordered_centerline)})")


In [ ]:

# 7. Plot
plt.figure(figsize=(10,10))
plt.imshow(img, cmap='gray')
plt.plot(ordered_centerline[:,0], ordered_centerline[:,1], 'r-', linewidth=1)
plt.title('Extracted Ordered Centerline')
plt.gca().invert_yaxis()
plt.show()
